In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
from os.path import abspath
import re
import os
import matplotlib.image
from functools import partial
from IPython.display import Image
from sklearn import linear_model
from statistics import median 
import plotly.offline as py
import seaborn as sns
import plotly.figure_factory as ff
import pingouin as pg

WEIGHT = 'WTPERFIN'

Regression for Census Division & Incomes

In [ ]:
incCensus = pd.read_excel('UnitInStructure.xlsx', sheet_name = 'linearRegression',header =None,index_col=0)
incCensus.reset_index(inplace = True)

In [ ]:
incCensus

In [ ]:
incCensus.loc[14,0] = 162500

In [ ]:
#incCensus

In [ ]:
arr = np.zeros(((len(incCensus.index)-1)*(len(incCensus.columns)-1), 3))
print(arr.shape)
incCensus.iloc[1,0]

In [ ]:
for i in range(len(incCensus.index)-1):
    for j in range(len(incCensus.columns)-1):
        arr[j+i*(len(incCensus.columns)-1)][0] = incCensus.iloc[i+1,0]
        arr[j+i*(len(incCensus.columns)-1)][1] = incCensus.iloc[0,j+1]
        arr[j+i*(len(incCensus.columns)-1)][2] = incCensus.iloc[i+1,j+1]


In [ ]:
arr
linPre = pd.DataFrame(arr,columns=['HHIncomes', 'CensusDivisions', 'MUDsShares'])
linPre
linPre_2 = linPre

In [ ]:
import statsmodels.api as sm
model = sm.OLS(linPre[['MUDsShares']], linPre[['HHIncomes','CensusDivisions']])
results = model.fit()
print(results.summary())

In [ ]:
# We have a categorical variable 'Census Divisions' which has 9 values, so we should only be using 8 dummy variables. 
# 'Pacific Division' becomes the reference category, in the case that the other dummy variables are all 0, 
# then by default the variable would be 'Pacific Division'.
division_list = ['NewEngland','MiddleAtlantic','EastNorthCentral','WestNorthCentral',\
                 'SouthAtlantic','EastSouthCentral','WestSouthCentral','Mountain']
for i, item in enumerate(division_list):
    linPre_2[item] = np.where(linPre_2['CensusDivisions']== i+1, 0, 1)

In [ ]:
#linPre_2

In [ ]:
# reg = linear_model.LinearRegression()
# reg.fit(linPre[['Incomes', 'Census Divisions']],linPre[['MUDs Shares']])
# print(reg.coef_)
# print(reg.intercept_)

In [ ]:
#x0.head()

In [ ]:
# x0 = trip17_MUD_C.loc[:,["HHIncomes","NewEngland","MiddleAtlantic","EastNorthCentral","WestNorthCentral","SouthAtlantic","EastSouthCentral","WestSouthCentral","Mountain"]]
# x0 = sm.tools.add_constant(pd.DataFrame(x0).T)
# results_3.predict(x0)


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
model2 = smf.ols(formula = "MUDsShares ~ HHIncomes + NewEngland+MiddleAtlantic+EastNorthCentral+\
                 WestNorthCentral+SouthAtlantic+EastSouthCentral+WestSouthCentral+Mountain+\
                 HHIncomes:NewEngland+HHIncomes:MiddleAtlantic+HHIncomes:EastNorthCentral+HHIncomes:WestNorthCentral+\
                 HHIncomes:SouthAtlantic+HHIncomes:EastSouthCentral+HHIncomes:WestSouthCentral+HHIncomes:Mountain",data=linPre_2)
results_3 = model2.fit()
#print(results_3.summary())
#print('p-values are: \n',results_3.pvalues)
# do either
print(results_3.summary().as_latex())
with open('summary_with_interaction_terms.txt', 'w') as fh:
    fh.write(results_3.summary().as_latex())
# alternatively
# for table in results.summary().tables:
#     print(table.as_latex_tabular())

In [ ]:

import statsmodels.api as sm
X = sm.add_constant(linPre_2[['HHIncomes']+division_list])
model = sm.OLS(linPre_2[['MUDsShares']], X)
results_2 = model.fit()
#print(results_2.summary())
print(results_2.summary().as_latex())
with open('summary_no_interaction_terms.txt', 'w') as fh:
    fh.write(results_2.summary().as_latex())
#print('p-values are: \n',results_2.pvalues)

In [ ]:
pg.corr(x=linPre_2['HHIncomes'], y=linPre_2['NewEngland'])

In [ ]:
pg.pairwise_corr(linPre_2).sort_values(by=['p-unc'])[['X', 'Y', 'n', 'r', 'p-unc']].head()

In [ ]:
linPre_2

In [ ]:
linPre_2_2 = linPre_2.drop(columns='CensusDivisions')
linPre_2_2.head()

In [ ]:
linPre_2_2.corr().round(2)

In [ ]:
corrs = linPre_2_2.corr()
mask = np.zeros_like(corrs)
mask[np.triu_indices_from(mask)] = True
plt.figure(figsize=(20,10))
sns.heatmap(corrs, cmap='Spectral_r', mask=mask, square=True, vmin=-.8, vmax=.3)
plt.title('Correlation matrix')

In [ ]:
# pers17 = pd.read_csv('NHTS-daily travel patterns/Csv/perpub.csv')

In [ ]:
# pers17.head()

In [ ]:
#veh17 = pd.read_csv('NHTS-daily travel patterns/Csv/vehpub.csv')
#veh17.head()

In [ ]:
trip17 = pd.read_csv('trippub.csv')
trip17.head()

In [ ]:
type(trip17.loc[0,'CENSUS_D'])

pick out only one primary vehicle for each household to generate the sub-dataset of original trip data.

In [ ]:
trip17_1pv = trip17.groupby(['HOUSEID','VEHID'], as_index=False)['VMT_MILE'].sum()

In [ ]:
trip17_process=trip17_1pv.groupby(['HOUSEID']).agg(list)

In [ ]:
trip17_process['freq_car_index']=trip17_process.agg(lambda x: x['VEHID'][np.argmax(x['VMT_MILE'])],axis=1)

In [ ]:
trip17_process=trip17_process.reset_index()

In [ ]:
trip17_process.head()

In [ ]:
trip17.head()

In [ ]:
trip17_final=trip17.merge(trip17_process[['HOUSEID','freq_car_index']],on=['HOUSEID'],how='left')

In [ ]:
trip17_final['selected_car']=trip17_final['freq_car_index']==trip17_final['VEHID']

In [ ]:
trip17_final[['freq_car_index','VEHID','selected_car']].head()

In [ ]:
trip17_final = trip17_final[trip17_final['selected_car'] == True]

In [ ]:
trip17_final.head()

In [ ]:
trip_inter = trip17_final.groupby(['HOUSEID','VEHID','HHFAMINC','CENSUS_D'], as_index=False)['VMT_MILE'].sum()

In [ ]:
#trip_inter.head()

In [ ]:
for i, item in enumerate(division_list):
    trip_inter[item] = np.where(trip_inter['CENSUS_D']== i+1, 0, 1)
#trip_inter.head()

In [ ]:
trip_inter.loc[:, 'const'] = 1

In [ ]:
income = (5000, 12500, 20000, 30000, 40000, 62500, 87500,112500, 137500, 175000, 250000) 
#trip17_MUD_C.loc[:, 'MUDs_Predict'] = 0
trip_inter.loc[:, 'HHIncomes'] = 0
for i in range(11):
    trip_inter['HHIncomes'].mask(trip_inter['HHFAMINC'] == i+1, income[i], inplace=True)
trip_inter[trip_inter['HHIncomes'] == 0] = median(income)

In [ ]:
Xnew = [trip_inter['const'],trip_inter['HHIncomes'],trip_inter[division_list[0]],trip_inter[division_list[1]],\
            trip_inter[division_list[2]],trip_inter[division_list[3]], trip_inter[division_list[4]],\
            trip_inter[division_list[5]], trip_inter[division_list[6]], trip_inter[division_list[7]]]
   

In [ ]:
Xnew=np.array(Xnew)

In [ ]:
Xnew.shape

In [ ]:
X = sm.add_constant(Xnew)

In [ ]:
X.shape

In [ ]:
results_2.predict(X[:,132])

In [ ]:
trip_inter.loc[:, 'MUDs_Predict'] = 0
for i in range(len(trip_inter)):
    trip_inter.loc[i,'MUDs_Predict'] = results_2.predict(X[:,i])[0]

In [ ]:
#trip_inter.head()

In [ ]:
## Predict corresponding MUDs share for trip takers
## merge MUD or others from AHS with the share of MUD in different CENSUS DIVISION

random_lists = []
for i in range(len(trip_inter)):
    if trip_inter.loc[i,'MUDs_Predict'] >= 1:
        #print(i)
        random_lists.append(1)
    elif trip_inter.loc[i,'MUDs_Predict'] > 0 :
        #print(i)
        random_lists.append(np.random.choice([1, 0], p=[trip_inter.loc[i,'MUDs_Predict'], 1-trip_inter.loc[i,'MUDs_Predict']]))
    else:
        random_lists.append(0)
trip_inter['MUDs_Predict_OW'] = random_lists



In [ ]:
random_lists = []
for i in range(len(trip_inter)):
    if trip_inter.loc[i,'MUDs_Predict'] >= 1:
        #print(i)
        random_lists.append(1)
    elif trip_inter.loc[i,'MUDs_Predict'] > 0 :
        #print(i)
        random_lists.append(np.random.choice([1, 0], p=[trip_inter.loc[i,'MUDs_Predict'], 1-trip_inter.loc[i,'MUDs_Predict']]))
    else:
        random_lists.append(0)
trip_inter['MUDs_Predict_OW_1'] = random_lists




In [ ]:
trip_inter['MUDs_Predict_OW'].value_counts()

In [ ]:
###### Multiple times (9 times) of simulation with the randomized assignment
count = 2
while (count < 10):
    random_lists = []
    for i in range(len(trip_inter)):
        if trip_inter.loc[i,'MUDs_Predict'] >= 1:
            #print(i)
            random_lists.append(1)
        elif trip_inter.loc[i,'MUDs_Predict'] > 0 :
            #print(i)
            random_lists.append(np.random.choice([1, 0], p=[trip_inter.loc[i,'MUDs_Predict'], 1-trip_inter.loc[i,'MUDs_Predict']]))
        else:
            random_lists.append(0)
    trip_inter['MUDs_Predict_OW_'+str(count)] = random_lists
    count += 1
    

In [ ]:
MUDs_no_lists = []
MUDs_no_lists.append(trip_inter['MUDs_Predict_OW'].sum())
for count in range(1,10):
    MUDs_no_lists.append(trip_inter['MUDs_Predict_OW_'+str(count)].sum())

In [ ]:
MUDs_no_lists

In [ ]:
###### (Expanding) Multiple times (90 times) of simulation with the randomized assignment
count = 10
while (count < 100):
    random_lists = []
    for i in range(len(trip_inter)):
        if trip_inter.loc[i,'MUDs_Predict'] >= 1:
            #print(i)
            random_lists.append(1)
        elif trip_inter.loc[i,'MUDs_Predict'] > 0 :
            #print(i)
            random_lists.append(np.random.choice([1, 0], p=[trip_inter.loc[i,'MUDs_Predict'], 1-trip_inter.loc[i,'MUDs_Predict']]))
        else:
            random_lists.append(0)
    trip_inter['MUDs_Predict_OW_'+str(count)] = random_lists
    MUDs_no_lists.append(trip_inter['MUDs_Predict_OW_'+str(count)].sum())
    count += 1
    

In [ ]:
trip_inter.head()

In [ ]:
# the first simulation
simu = trip_inter.loc[:,'MUDs_Predict_OW':'MUDs_Predict_OW_1']

In [ ]:
# 100 simulation
simu = trip_inter.loc[:,'MUDs_Predict_OW':'MUDs_Predict_OW_99']

In [ ]:
simu.insert(loc = 0, 
          column = 'HOUSEID', 
          value = trip_inter['HOUSEID']) 

In [ ]:
#simu.head()

In [ ]:
# simu.to_csv('new_simulation_result_income_162500.csv',index=False)

In [ ]:
trip17_pred=trip17_final.merge(simu,on=['HOUSEID'],how='left')

In [ ]:
trip17_pred.head()

check each division has approximate MUDs share from the simulation

In [ ]:
division_list_9 = ['New England','Middle Atlantic','East North Central','West North Central',\
                 'South Atlantic','East South Central','West South Central','Mountain','Pacific']

In [ ]:
trip17_pred_uni = trip17_pred.groupby(['HOUSEID','CENSUS_D'], as_index=False)['MUDs_Predict_OW'].mean()

In [ ]:
#trip17_pred_uni.head()

In [ ]:
division = trip17_pred_uni[trip17_pred_uni['CENSUS_D'] == 0+3]
division['MUDs_Predict_OW'].value_counts()

In [ ]:
MUD_share = []
for i in range(9):
    division = trip17_pred_uni[trip17_pred_uni['CENSUS_D'] == i+1]
    temp = division['MUDs_Predict_OW'].value_counts().tolist()[1]/sum(division['MUDs_Predict_OW'].value_counts().tolist())
    temp = round(temp, 3)
    MUD_share.append(temp)
    print('MUDs share of',division_list_9[i],'division is: ',MUD_share[i])

In [ ]:
fig, ax = plt.subplots(figsize=(8,5)) 
ax.bar(division_list_9,MUD_share)
ax.set_title('MUDs Share per Census Division')

for tick in ax.get_xticklabels():
    tick.set_rotation(45)
#ax.set_rotation(rotation=45)

## Space-Time Path

In [ ]:
trip17_pred.head(20)

In [ ]:
trip17_pred_MUD = trip17_pred[trip17_pred['MUDs_Predict_OW'] == 1]

In [ ]:
trip17_pred_MUD.head(20)

In [ ]:
part_trip17_pred_MUD = trip17_pred_MUD[['HOUSEID','PERSONID','VEHID','STRTTIME','ENDTIME','TRVLCMIN','TRPMILES','WHYTRP1S']]
part_trip17_pred_MUD.head()

In [ ]:
part_trip17_pred_MUD['WHYTRP1S'] = np.where(part_trip17_pred_MUD['WHYTRP1S']== 1, 'Home', 
                                             np.where(part_trip17_pred_MUD['WHYTRP1S']== 10, 'Work','Public'))

part_trip17_pred_MUD.head()

In [ ]:
part_trip17_pred_MUD.head(30)

In [ ]:
# case: HOUSEID == 30000161 [can be taken as housewife]
trip17_pred_MUD_case_1 = part_trip17_pred_MUD[part_trip17_pred_MUD['HOUSEID'] == 30000161]
trip17_pred_MUD_case_1

In [ ]:
# case: HOUSEID == 30000265 [can be taken as normal worker]
trip17_pred_MUD_case_2 = part_trip17_pred_MUD[part_trip17_pred_MUD['HOUSEID'] == 30000265]
trip17_pred_MUD_case_2

In [ ]:
trip17_MUD_C = trip17_final
division_list = ['NewEngland','MiddleAtlantic','EastNorthCentral','WestNorthCentral',\
                 'SouthAtlantic','EastSouthCentral','WestSouthCentral','Mountain']
for i, item in enumerate(division_list):
    trip17_MUD_C[item] = np.where(trip17_MUD_C['CENSUS_D']== i+1, 0, 1)
trip17_MUD_C.head()

In [ ]:
# Normolize income with median value
income = (5000, 12500, 20000, 30000, 40000, 62500, 87500,112500, 137500, 175000, 250000) 
#trip17_MUD_C.loc[:, 'MUDs_Predict'] = 0
trip17_MUD_C.loc[:, 'HHIncomes'] = 0
for i in range(11):
    trip17_MUD_C['HHIncomes'].mask(trip17_MUD_C['HHFAMINC'] == i+1, income[i], inplace=True)
trip17_MUD_C[trip17_MUD_C['HHIncomes'] == 0] = median(income)
# for i in range(len(trip17_MUD_C)):
#     # get predicted MUDs share for each trip taker
#     Xnew = [trip17_MUD_C.loc[i, 'HHIncomes'],trip17_MUD_C.loc[i, division_list[0]],trip17_MUD_C.loc[i, division_list[1]],\
#             trip17_MUD_C.loc[i, division_list[2]],trip17_MUD_C.loc[i, division_list[3]], trip17_MUD_C.loc[i, division_list[4]],\
#             trip17_MUD_C.loc[i, division_list[5]], trip17_MUD_C.loc[i, division_list[6]], trip17_MUD_C.loc[i, division_list[7]]]
#     trip17_MUD_C.loc[i,'MUDs_Predict'] = results_2.predict(Xnew)[0]

In [ ]:
Xnew = [trip17_MUD_C['HHIncomes'],trip17_MUD_C[division_list[0]],trip17_MUD_C[division_list[1]],\
            trip17_MUD_C[division_list[2]],trip17_MUD_C[division_list[3]], trip17_MUD_C[division_list[4]],\
            trip17_MUD_C[division_list[5]], trip17_MUD_C[division_list[6]], trip17_MUD_C[division_list[7]]]
   

In [ ]:
x0=np.array(x0)

In [ ]:
x0.shape

In [ ]:
Xnew=np.array(Xnew)

In [ ]:
results_3.predict(120000,1,1,1,1,1,1,0,1)

In [ ]:
prediction = results_3.get_prediction(exog=dict(x0[0,:])) 

In [ ]:
for i in range(len(trip17_MUD_C)):
    trip17_MUD_C.loc[i,'MUDs_Predict'] = results_3.predict(x0[:,i])[0]

In [ ]:
x0 = trip17_MUD_C.loc[:,("HHIncomes",division_list[0],division_list[1],division_list[2],division_list[3],\
                         division_list[4],division_list[5],division_list[6],division_list[7])]
x0 = sm.tools.add_constant(pd.DataFrame(x0).T)
results_3.predict(x0)

In [ ]:
trip17_MUD_C.loc[:, 'MUDs_Predict'] = ls

In [ ]:
len(trip17_MUD_C)

In [ ]:
paci = trip17_MUD_C[trip17_MUD_C['CENSUS_D']==9]
paci['MUDs_Predict'].head(20)

In [ ]:
paci['MUDs_Predict']

In [ ]:
random_lists = []
for i in range(len(paci)):
    if paci.loc[i,'MUDs_Predict'] >= 1:
        random_lists.append(1)
    elif paci.loc[i,'MUDs_Predict'] > 0 :
        #print(i)
        random_lists.append(np.random.choice([1, 0], p=[paci.loc[i,'MUDs_Predict'], 1-paci.loc[i,'MUDs_Predict']]))
    else:
        random_lists.append(0)
paci['MUDs_Predict_OW'] = random_lists


In [ ]:
len(trip17_MUD_C[trip17_MUD_C['CENSUS_D']==9])

In [ ]:
trip17_MUD_C.head(20)

In [ ]:
for i in range(len(trip17_MUD_C)):
    trip17_MUD_C.loc[i,'MUDs_Predict'] = results_2.predict(Xnew[:,i])[0]

In [ ]:
trip17_MUD_C.loc[:,'MUDs_Predict'].fillna(0)

In [ ]:
## Predict corresponding MUDs share for trip takers
## merge MUD or others from AHS with the share of MUD in different CENSUS DIVISION

random_lists = []
for i in range(len(trip17_MUD_C)):
    if trip17_MUD_C.loc[i,'MUDs_Predict'] >= 1:
        #print(i)
        random_lists.append(1)
    elif trip17_MUD_C.loc[i,'MUDs_Predict'] > 0 :
        #print(i)
        random_lists.append(np.random.choice([1, 0], p=[trip17_MUD_C.loc[i,'MUDs_Predict'], 1-trip17_MUD_C.loc[i,'MUDs_Predict']]))
    else:
        random_lists.append(0)
trip17_MUD_C['MUDs_Predict_OW'] = random_lists


### Multiple simulations to convince the robustness of the ranmized assignment

In [ ]:
###### Multiple times (9 times) of simulation with the randomized assignment
count = 1
while (count < 10):
    random_lists = []
    for i in range(len(trip17_MUD_C)):
        if trip17_MUD_C.loc[i,'MUDs_Predict'] >= 1:
            #print(i)
            random_lists.append(1)
        elif trip17_MUD_C.loc[i,'MUDs_Predict'] > 0 :
            #print(i)
            random_lists.append(np.random.choice([1, 0], p=[trip17_MUD_C.loc[i,'MUDs_Predict'], 1-trip17_MUD_C.loc[i,'MUDs_Predict']]))
        else:
            random_lists.append(0)
    trip17_MUD_C['MUDs_Predict_OW_'+str(count)] = random_lists
    count += 1

In [ ]:
trip17_MUD_C['MUDs_Predict_OW'].sum()

In [ ]:
MUDs_no_lists = []
MUDs_no_lists.append(trip17_MUD_C['MUDs_Predict_OW'].sum())
for count in range(1,10):
    MUDs_no_lists.append(trip17_MUD_C['MUDs_Predict_OW_'+str(count)].sum())

In [ ]:
print(median(MUDs_no_lists))
import statistics
statistics.stdev(MUDs_no_lists)

In [ ]:
print(median(MUDs_no_lists))
statistics.stdev(MUDs_no_lists)

In [ ]:
sum(MUDs_no_lists)/(len(MUDs_no_lists))

In [ ]:
simulation_result = trip17_MUD_C.loc[:,'MUDs_Predict_OW':'MUDs_Predict_OW_99']

In [ ]:
simulation_result.head()

In [ ]:
simulation_result.to_csv('simulation_result.csv',index=False)

In [ ]:
df = pd.read_csv('simulation_result.csv')

In [ ]:
df.head()

In [ ]:
###### (Expanding) Multiple times (90 times) of simulation with the randomized assignment
count = 10
while (count < 100):
    random_lists = []
    for i in range(len(trip17_MUD_C)):
        if trip17_MUD_C.loc[i,'MUDs_Predict'] >= 1:
            #print(i)
            random_lists.append(1)
        elif trip17_MUD_C.loc[i,'MUDs_Predict'] > 0 :
            #print(i)
            random_lists.append(np.random.choice([1, 0], p=[trip17_MUD_C.loc[i,'MUDs_Predict'], 1-trip17_MUD_C.loc[i,'MUDs_Predict']]))
        else:
            random_lists.append(0)
    trip17_MUD_C['MUDs_Predict_OW_'+str(count)] = random_lists
    MUDs_no_lists.append(trip17_MUD_C['MUDs_Predict_OW_'+str(count)].sum())
    count += 1

In [ ]:
tod_trip17 = trip17

In [ ]:
tod_trip17.sort_values(by='STRTTIME', ascending=True, na_position='first', inplace=True)

In [ ]:
tod_trip17.head()

### previous predictions exclusively considering income

In [ ]:
## Predict corresponding MUDs share for trip takers
income = (5000, 12500, 20000, 30000, 40000, 62500, 87500,112500, 137500, 175000, 250000) 
trip17_MUD = trip17
trip17_MUD.loc[:, 'MUDs_Predict'] = 0
trip17_MUD.loc[:, 'HHIncomes'] = 0
for i in range(len(trip17_MUD)):
    if trip17_MUD.loc[i, 'HHFAMINC'] == 1:
        trip17_MUD.loc[i, 'HHIncomes'] = 5000
    if trip17_MUD.loc[i, 'HHFAMINC'] == 2:
        trip17_MUD.loc[i, 'HHIncomes'] = 12500
    if trip17_MUD.loc[i, 'HHFAMINC'] == 3:
        trip17_MUD.loc[i, 'HHIncomes'] = 20000
    if trip17_MUD.loc[i, 'HHFAMINC'] == 4:
        trip17_MUD.loc[i, 'HHIncomes'] = 30000
    if trip17_MUD.loc[i, 'HHFAMINC'] == 5:
        trip17_MUD.loc[i, 'HHIncomes'] = 40000
    if trip17_MUD.loc[i, 'HHFAMINC'] == 6:
        trip17_MUD.loc[i, 'HHIncomes'] = 62500
    if trip17_MUD.loc[i, 'HHFAMINC'] == 7:
        trip17_MUD.loc[i, 'HHIncomes'] = 87500
    if trip17_MUD.loc[i, 'HHFAMINC'] == 8:
        trip17_MUD.loc[i, 'HHIncomes'] = 112500
    if trip17_MUD.loc[i, 'HHFAMINC'] == 9:
        trip17_MUD.loc[i, 'HHIncomes'] = 137500
    if trip17_MUD.loc[i, 'HHFAMINC'] == 10:
        trip17_MUD.loc[i, 'HHIncomes'] = 175000
    if trip17_MUD.loc[i, 'HHFAMINC'] == 11:
        trip17_MUD.loc[i, 'HHIncomes'] = 250000
    else:
        trip17_MUD.loc[i, 'HHIncomes'] = median(income)
    #trip17_MUD.loc[i,'MUD_OW'] = reg.predict([[87500,2]])[0][0]
    trip17_MUD.loc[i,'MUDs_Predict'] = results.predict([trip17_MUD.loc[i, 'HHIncomes'],trip17_MUD.loc[i, 'CENSUS_D']])[0]

In [ ]:
trip17_MUD

In [ ]:
Prob_MUD = np.array([0.326,0.341,0.211,0.186,0.213,0.154,0.206,0.197,0.286])

In [ ]:
for i in range(len(trip17_MUD)):
    trip17_MUD.loc[i,'MUDs_Predict'] = reg.predict([[trip17_MUD.loc[i, 'HHIncomes'],trip17_MUD.loc[i, 'CENSUS_D']]])[0][0]
    if trip17_MUD.loc[i,'MUDs_Predict'] < 0:
        trip17_MUD.loc[i,'MUDs_Predict'] = Prob_MUD[trip17_MUD.loc[i,'CENSUS_D']-1]

In [ ]:
## Predict corresponding MUDs share for trip takers
## merge MUD or others from AHS with the share of MUD in different CENSUS DIVISION
import time
t1= time.time()
trip17_MUD = trip17
trip17_MUD.loc[:, 'MUDs_Predict_OW'] = 0

mudsPredict= []
#count = 11
#while (count > 0):
for i in range(len(trip17_MUD)): 
    trip17_MUD.loc[i,'MUDs_Predict_OW'] = np.random.choice([1, 0], p=[trip17_MUD.loc[i,'MUDs_Predict'],
                                                          1-trip17_MUD.loc[i,'MUDs_Predict']])
mudsPredict.append(trip17_MUD['MUDs_Predict_OW'].sum())
#count-=1
    
t2= time.time()
t=t2-t1
print(t)
print(mudsPredict)

### previous predictions with fixed MUDs share by CENSUN DIVISION

In [ ]:
## merge MUD or others from AHS with the share of MUD in different CENSUS DIVISION
import time
t1= time.time()
Prob_MUD = np.array([0.326,0.341,0.211,0.186,0.213,0.154,0.206,0.197,0.286])
trip17_MUD = trip17
trip17_MUD.loc[:, 'MUDs_OW'] = 0

muds= []
count = 11
#while (count > 0):
for i in range(len(trip17_MUD)): 
    trip17_MUD.loc[i,'MUDs_OW'] = np.random.choice([1, 0], p=[Prob_MUD[trip17_MUD.loc[i,'CENSUS_D']-1],
                                                          1-Prob_MUD[trip17_MUD.loc[i,'CENSUS_D']-1]])
muds.append(trip17_MUD['MUD_OW'].sum())
count-=1
    
t2= time.time()
t=t2-t1
print(t)

t2= time.time()
t=t2-t1
print(t)
print(muds)

In [ ]:
x = np.random.choice([1, 0], p=[0.2, 0.8])

print(x)

In [ ]:
# trip17_MUD_True = trip17_MUD[trip17_MUD['MUD_OW'] == 1]
# trip17_MUD_False = trip17_MUD[trip17_MUD['MUD_OW'] == 0]

trip17_MUD_True_C = trip17_MUD_C[trip17_MUD_C['MUDs_Predict_OW'] == 1]
trip17_MUD_False_C = trip17_MUD_C[trip17_MUD_C['MUDs_Predict_OW'] == 0]

In [ ]:
trip17_MUD.head()

In [ ]:
### new Trip Distance Distribution with new categrarical CENSUS DIVISIONS using Dummy Variables
plt.style.use("fivethirtyeight")

trip17_MUD_True_C1 = trip17_MUD_True_C[trip17_MUD_True_C['TRPMILES']>0]

trip17_MUD_True_C1 = trip17_MUD_True_C1[
    np.abs(trip17_MUD_True_C1.TRPMILES - trip17_MUD_True_C1.TRPMILES.mean()) <= (3*trip17_MUD_True_C1.TRPMILES.std())
]
trip17_MUD_True_C1 = trip17_MUD_True_C1.groupby('TRPMILES').WTTRDFIN.sum().reset_index(name ='count')
trip17_MUD_True_C1 = trip17_MUD_True_C1.groupby(pd.cut(trip17_MUD_True_C1.TRPMILES, bins=np.arange(trip17_MUD_True_C1.TRPMILES.min(), trip17_MUD_True_C1.TRPMILES.max(), step=10),include_lowest=True)).sum()
trip17_MUD_True_C1 = trip17_MUD_True_C1.head(22)

trip17_MUD_False_C1 = trip17_MUD_False_C[trip17_MUD_False_C['TRPMILES']>0]

trip17_MUD_False_C1 = trip17_MUD_False_C1[
    np.abs(trip17_MUD_False_C1.TRPMILES - trip17_MUD_False_C1.TRPMILES.mean()) <= (3*trip17_MUD_False_C1.TRPMILES.std())
]
trip17_MUD_False_C1 = trip17_MUD_False_C1.groupby('TRPMILES').WTTRDFIN.sum().reset_index(name ='count')
trip17_MUD_False_C1 = trip17_MUD_False_C1.groupby(pd.cut(trip17_MUD_False_C1.TRPMILES, bins=np.arange(trip17_MUD_False_C1.TRPMILES.min(), trip17_MUD_False_C1.TRPMILES.max(), step=10),include_lowest=True)).sum()
trip17_MUD_False_C1 = trip17_MUD_False_C1.head(22)



width = 0.35       # the width of the bars: can also be len(x) sequence
plt.figure(figsize =(20,12))

plt.bar(TRPMILES, trip17_MUD_True_C1['count'], width, label='MUDs')
plt.bar(TRPMILES, trip17_MUD_False_C1['count'], width, label='Others',bottom=np.array(trip17_MUD_True_1['count']) )

plt.xlabel('TRPMILES(miles)')
plt.ylabel('Counts of trips')
plt.title('Trip distance in miles')
plt.legend(loc='upper right')
plt.xticks(rotation=45)

plt.show()

In [ ]:
simu_re = pd.read_csv('simulation_result.csv')

In [ ]:
simu_re.head()

In [ ]:
simu_re.shape

In [ ]:
trip17 = pd.read_csv('trippub.csv')
trip17.head()

In [ ]:
trip17.shape

In [ ]:
trip17_MUD = trip17.join(simu_re, how="inner")
trip17_MUD.shape

In [ ]:
trip17_MUD.head()

In [ ]:
# Drop all zero values of trip dataset
trip17_MUD = trip17_MUD.loc[trip17_MUD.WTTRDFIN > 0]

# Remove outliers which are not within 3 standard deviations from mean of trip dataset
trip17_MUD = trip17_MUD[
    np.abs(trip17_MUD.WTTRDFIN - trip17_MUD.WTTRDFIN.mean()) <= (3*trip17_MUD.WTTRDFIN.std())
]

100 runs of the semi-randomized process creating the MUDs and single family housing categories reporting:

    descriptive statistics like daily vehicle miles traveled, percentage of time spend at home
    distributions of vehicle miles traveled per time of day, etc. 

time of day analysis of travel patterns similar to my Transportation Research record paper for each housing category (single family housing and MUDs) as well as for different census divisions

In [ ]:
trip17_MUD.shape

In [ ]:
466346/(466346+212062+120576+45693+41295+11148+2518+1474+671+188+90+29+28+4)

In [ ]:
trip17_MUD['VEHID'].value_counts()

In [ ]:
### Trip distance in miles for personally driven vehicle trips
### new Trip Distance Distribution with new categrarical CENSUS DIVISIONS using Dummy Variables
plt.style.use("seaborn")
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 16,
        }
trip17_MUD_vmt = trip17_MUD[trip17_MUD['VMT_MILE']>0]

In [ ]:
trip17_MUD_vmt.shape

Time of day process

In [ ]:
trip17_MUD_vmt.head(1)

In [ ]:
avg_vmt_T = [[0 for i in range(24)] for j in range(100)]
avg_vmt_F = [[0 for i in range(24)] for j in range(100)]
avg_vmt = [[0 for i in range(24)] for j in range(100)]

trip17_MUD_vmt_T = trip17_MUD_vmt[trip17_MUD_vmt['MUDs_Predict_OW'] == 1]
trip17_MUD_vmt_F = trip17_MUD_vmt[trip17_MUD_vmt['MUDs_Predict_OW'] == 0]

#trip17_MUD_C['HHIncomes'].mask(trip17_MUD_C['HHFAMINC'] == i+1, income[i], inplace=True)
for i in range(24):
    avg_vmt_T[0][i] = trip17_MUD_vmt_T[(trip17_MUD_vmt_T['STRTTIME'] >= 100*i) & (trip17_MUD_vmt_T['STRTTIME']< 100*(i+1))].VMT_MILE.mean()
    avg_vmt_F[0][i] = trip17_MUD_vmt_F[(trip17_MUD_vmt_F['STRTTIME'] >= 100*i) & (trip17_MUD_vmt_F['STRTTIME']< 100*(i+1))].VMT_MILE.mean()
    avg_vmt[0][i] = trip17_MUD_vmt[(trip17_MUD_vmt['STRTTIME'] >= 100*i) & (trip17_MUD_vmt['STRTTIME']< 100*(i+1))].VMT_MILE.mean()
    

In [ ]:
trip17_MUD_vmt.head()

In [ ]:
for i in range(1,100):
    trip17_MUD_vmt_T = trip17_MUD_vmt[trip17_MUD_vmt['MUDs_Predict_OW_'+str(i)] == 1]
    trip17_MUD_vmt_F = trip17_MUD_vmt[trip17_MUD_vmt['MUDs_Predict_OW_'+str(i)] == 0]
    for j in range(24):
        avg_vmt_T[i][j] = trip17_MUD_vmt_T[(trip17_MUD_vmt_T['STRTTIME'] >= 100*j) & (trip17_MUD_vmt_T['STRTTIME']< 100*(j+1))].VMT_MILE.mean()
        avg_vmt_F[i][j] = trip17_MUD_vmt_F[(trip17_MUD_vmt_F['STRTTIME'] >= 100*j) & (trip17_MUD_vmt_F['STRTTIME']< 100*(j+1))].VMT_MILE.mean()
        avg_vmt[i][j] = trip17_MUD_vmt[(trip17_MUD_vmt['STRTTIME'] >= 100*j) & (trip17_MUD_vmt['STRTTIME']< 100*(j+1))].VMT_MILE.mean()
    

In [ ]:
col_list = range(24)
df_avg_vmt_T = pd.DataFrame(data=avg_vmt_T, columns=col_list)
df_avg_vmt_F = pd.DataFrame(data=avg_vmt_F, columns=col_list)
df_avg_vmt = pd.DataFrame(data=avg_vmt, columns=col_list)
df_avg_vmt_T

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(16, 12))
fig.suptitle('Time of Day Average VMT for personally driven vehicle trips (Mile)', fontsize=28)
 
# data required for plotting
all_data = [df_avg_vmt_T[i] for i in range(24)]
all_data_F = [df_avg_vmt_F[i] for i in range(24)]

# plot violin plot
axs[0][0].violinplot(all_data,
                  showmeans=False,
                  showmedians=True)
axs[0][0].set_title('Violin plot',fontdict=font)

# plot box plot
axs[0][1].boxplot(all_data)
axs[0][1].set_title('Box plot',fontdict=font)

# plot violin plot
axs[1][0].violinplot(all_data_F,
                  showmeans=False,
                  showmedians=True)
#axs[1][0].set_title('Violin plot')

# plot box plot
axs[1][1].boxplot(all_data_F)
#axs[1][1].set_title('Box plot')

# adding horizontal grid lines
for ax in axs[0]:
    ax.yaxis.grid(True)
    ax.set_xticks([y + 1 for y in range(len(all_data))])
    
    #ax.set_xlabel('Time of Day')
axs[0][0].set_ylabel("VMT of MUDs' Residents",fontdict=font)

for ax in axs[1]:
    ax.yaxis.grid(True)
    ax.set_xticks([y + 1 for y in range(len(all_data_F))])
    
    ax.set_xlabel('Time of Day',fontdict=font)
axs[1][0].set_ylabel("VMT of Others' Residents",fontdict=font)
    
# add x-tick labels


plt.setp(axs, xticks=[y + 1 for y in range(len(all_data))],
         xticklabels=range(1,25))

plt.show()

In [ ]:
tod = []
for i in range(24):
    tod.append(str(i) + ':00 to ' + str(i+1) + ':00')

col_list = tod
stat_avg_vmt = pd.DataFrame(columns=col_list,index = ['mean of MUDs', 'std of MUDs', 'mean of Others','std of Others','mean of Alls','std of Alls'])
stat_avg_vmt

In [ ]:
for i in range(24):
    stat_avg_vmt.iloc[0,i] = df_avg_vmt_T[i].mean()
    stat_avg_vmt.iloc[1,i] = df_avg_vmt_T[i].std()
    stat_avg_vmt.iloc[2,i] = df_avg_vmt_F[i].mean()
    stat_avg_vmt.iloc[3,i] = df_avg_vmt_F[i].std()
    stat_avg_vmt.iloc[4,i] = df_avg_vmt[i].mean()
    stat_avg_vmt.iloc[5,i] = df_avg_vmt[i].std()
stat_avg_vmt   

In [ ]:
tod = []
for i in range(24):
    tod.append(str(i) + ':00 to ' + str(i+1) + ':00')

plt.figure(figsize =(20,12))

plt.plot(tod, avg_vmt_T[0], label='MUDs');
plt.plot(tod, avg_vmt_F[0], label='Others');
#plt.plot(tod, avg_vmt[0], label='Alls');

plt.xlabel('Time of day', fontdict=font)
plt.ylabel('Average VMT (Miles)', fontdict=font)
plt.title('Average VMTs of predicted MUDs', fontsize=28)
plt.legend(loc='upper right')
plt.xticks(rotation=45)

plt.show()

In [ ]:
trip17_MUD_vmt_T = trip17_MUD_vmt[trip17_MUD_vmt['MUDs_Predict_OW'] == 1]
trip17_MUD_vmt_F = trip17_MUD_vmt[trip17_MUD_vmt['MUDs_Predict_OW'] == 0]
trip17_MUD_vmt_T = trip17_MUD_vmt_T[
    np.abs(trip17_MUD_vmt_T.VMT_MILE - trip17_MUD_vmt_T.VMT_MILE.mean()) <= (3*trip17_MUD_vmt_T.VMT_MILE.std())
]

trip17_MUD_vmt_F = trip17_MUD_vmt_F[
    np.abs(trip17_MUD_vmt_F.VMT_MILE - trip17_MUD_vmt_F.VMT_MILE.mean()) <= (3*trip17_MUD_vmt_F.VMT_MILE.std())
]
f, axes = plt.subplots(2, 1, figsize=(20, 16), sharex=True)
#plt.figure(figsize =(20,12))

sns.distplot(trip17_MUD_vmt_T['VMT_MILE'],ax=axes[0]);
axes[0].set(xlabel='VMT (mile)', ylabel="Density", title = 'VMT of MUDs')

sns.distplot(trip17_MUD_vmt_F['VMT_MILE'],ax=axes[1],color="g");
axes[1].set(xlabel='VMT (mile)', ylabel="Density", title = 'VMT of Others')
#axes.legend(loc='upper right');
f.suptitle('Distribution of Trip distance in miles for personally driven vehicle trips', fontsize=24);

In [ ]:

trip17_MUD_vmt_T = trip17_MUD_vmt[trip17_MUD_vmt['MUDs_Predict_OW'] == 1]
trip17_MUD_vmt_F = trip17_MUD_vmt[trip17_MUD_vmt['MUDs_Predict_OW'] == 0]


trip17_MUD_vmt_T = trip17_MUD_vmt_T[
    np.abs(trip17_MUD_vmt_T.VMT_MILE - trip17_MUD_vmt_T.VMT_MILE.mean()) <= (3*trip17_MUD_vmt_T.VMT_MILE.std())
]
trip17_MUD_vmt_T = trip17_MUD_vmt_T.groupby('VMT_MILE').WTTRDFIN.sum().reset_index(name ='count')
trip17_MUD_vmt_T = trip17_MUD_vmt_T.groupby(pd.cut(trip17_MUD_vmt_T.VMT_MILE, bins=np.arange(trip17_MUD_vmt_T.VMT_MILE.min(), trip17_MUD_vmt_T.VMT_MILE.max(), step=10),include_lowest=True)).sum()
trip17_MUD_vmt_T = trip17_MUD_vmt_T.append({'VMT_MILE': 0, 'count':0}, ignore_index=True)
trip17_MUD_vmt_T = trip17_MUD_vmt_T.append({'VMT_MILE': 0, 'count':0}, ignore_index=True)


trip17_MUD_vmt_F = trip17_MUD_vmt_F[
    np.abs(trip17_MUD_vmt_F.VMT_MILE - trip17_MUD_vmt_F.VMT_MILE.mean()) <= (3*trip17_MUD_vmt_F.VMT_MILE.std())
]
trip17_MUD_vmt_F = trip17_MUD_vmt_F.groupby('VMT_MILE').WTTRDFIN.sum().reset_index(name ='count')
trip17_MUD_vmt_F = trip17_MUD_vmt_F.groupby(pd.cut(trip17_MUD_vmt_F.VMT_MILE, bins=np.arange(trip17_MUD_vmt_F.VMT_MILE.min(), trip17_MUD_vmt_F.VMT_MILE.max(), step=10),include_lowest=True)).sum()

VMT_MILE = []
for i in range(10):
    VMT_MILE.append(str(10*i) + ' to ' + str(10*(i+1)))

x = np.arange(len(VMT_MILE))  # the label locations
width = 0.35       # the width of the bars: can also be len(x) sequence

fig, ax = plt.subplots(figsize = (16,8))

# rects1 = ax.bar(x - width/2, men_means, width, label='Men')
# rects2 = ax.bar(x - width/2, women_means, width, label='Women')

# plt.bar(VMT_MILE, trip17_MUD_vmt_T['count'],x - width/2, width, label='MUDs');
# plt.bar(VMT_MILE, trip17_MUD_vmt_F['count'],x + width/2, width, label='Others');

ax.bar(x - width/2, trip17_MUD_vmt_T['count'], width, label='MUDs');
ax.bar(x + width/2, trip17_MUD_vmt_F['count'], width, label='Others');

ax.set_xlabel('VMT_MILE(miles)', fontdict=font)
ax.set_ylabel('Counts of trips', fontdict=font)
ax.set_title('Distribution of counts of Trip distance in miles for personally driven vehicle trips', fontdict=font)
ax.legend(loc='upper right')
#ax.xticks(rotation=45)
ax.set_xticks(x)
ax.set_xticklabels(VMT_MILE)

plt.show()

# Analysis based on only one primary vehicle for each household

In [ ]:
trip17_MUD['VEHID'].value_counts()

In [ ]:
# filtering exclusively one primary vehicle for each household 
# by "vehid == 1"
trip17_MUD_1v = trip17_MUD[trip17_MUD['VEHID'] == 1]

In [ ]:
trip17_MUD_1v['VEHID'].value_counts()

In [ ]:
### Trip distance in miles for personally driven vehicle trips
### new Trip Distance Distribution with new categrarical CENSUS DIVISIONS using Dummy Variables
plt.style.use("seaborn")
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 16,
        }
trip17_MUD_1v_vmt = trip17_MUD_1v[trip17_MUD_1v['VMT_MILE']>0]

In [ ]:
avg_1v_vmt_T = [[0 for i in range(24)] for j in range(100)]
avg_1v_vmt_F = [[0 for i in range(24)] for j in range(100)]
avg_1v_vmt = [[0 for i in range(24)] for j in range(100)]

trip17_MUD_1v_vmt_T = trip17_MUD_1v_vmt[trip17_MUD_1v_vmt['MUDs_Predict_OW'] == 1]
trip17_MUD_1v_vmt_F = trip17_MUD_1v_vmt[trip17_MUD_1v_vmt['MUDs_Predict_OW'] == 0]

#trip17_MUD_C['HHIncomes'].mask(trip17_MUD_C['HHFAMINC'] == i+1, income[i], inplace=True)
for i in range(24):
    avg_1v_vmt_T[0][i] = trip17_MUD_1v_vmt_T[(trip17_MUD_1v_vmt_T['STRTTIME'] >= 100*i) & (trip17_MUD_1v_vmt_T['STRTTIME']< 100*(i+1))].VMT_MILE.mean()
    avg_1v_vmt_F[0][i] = trip17_MUD_1v_vmt_F[(trip17_MUD_1v_vmt_F['STRTTIME'] >= 100*i) & (trip17_MUD_1v_vmt_F['STRTTIME']< 100*(i+1))].VMT_MILE.mean()
    avg_1v_vmt[0][i] = trip17_MUD_1v_vmt[(trip17_MUD_1v_vmt['STRTTIME'] >= 100*i) & (trip17_MUD_1v_vmt['STRTTIME']< 100*(i+1))].VMT_MILE.mean()
    

In [ ]:
for i in range(1,100):
    trip17_MUD_1v_vmt_T = trip17_MUD_1v_vmt[trip17_MUD_1v_vmt['MUDs_Predict_OW_'+str(i)] == 1]
    trip17_MUD_1v_vmt_F = trip17_MUD_1v_vmt[trip17_MUD_1v_vmt['MUDs_Predict_OW_'+str(i)] == 0]
    for j in range(24):
        avg_1v_vmt_T[i][j] = trip17_MUD_1v_vmt_T[(trip17_MUD_1v_vmt_T['STRTTIME'] >= 100*j) & (trip17_MUD_1v_vmt_T['STRTTIME']< 100*(j+1))].VMT_MILE.mean()
        avg_1v_vmt_F[i][j] = trip17_MUD_1v_vmt_F[(trip17_MUD_1v_vmt_F['STRTTIME'] >= 100*j) & (trip17_MUD_1v_vmt_F['STRTTIME']< 100*(j+1))].VMT_MILE.mean()
        avg_1v_vmt[i][j] = trip17_MUD_1v_vmt[(trip17_MUD_1v_vmt['STRTTIME'] >= 100*j) & (trip17_MUD_1v_vmt['STRTTIME']< 100*(j+1))].VMT_MILE.mean()
    

In [ ]:
col_1v_list = range(24)
df_avg_1v_vmt_T = pd.DataFrame(data=avg_1v_vmt_T, columns=col_1v_list)
df_avg_1v_vmt_F = pd.DataFrame(data=avg_1v_vmt_F, columns=col_1v_list)
df_avg_1v_vmt = pd.DataFrame(data=avg_1v_vmt, columns=col_1v_list)
df_avg_1v_vmt_T

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(16, 12))
fig.suptitle('Time of Day Average VMT of trips for one primary vehicle per household (Mile)', fontsize=28)
 
# data required for plotting
all_data = [df_avg_1v_vmt_T[i] for i in range(24)]
all_data_F = [df_avg_1v_vmt_F[i] for i in range(24)]

# plot violin plot
axs[0][0].violinplot(all_data,
                  showmeans=False,
                  showmedians=True)
axs[0][0].set_title('Violin plot',fontdict=font)

# plot box plot
axs[0][1].boxplot(all_data)
axs[0][1].set_title('Box plot',fontdict=font)

# plot violin plot
axs[1][0].violinplot(all_data_F,
                  showmeans=False,
                  showmedians=True)
#axs[1][0].set_title('Violin plot')

# plot box plot
axs[1][1].boxplot(all_data_F)
#axs[1][1].set_title('Box plot')

# adding horizontal grid lines
for ax in axs[0]:
    ax.yaxis.grid(True)
    ax.set_xticks([y + 1 for y in range(len(all_data))])
    
    #ax.set_xlabel('Time of Day')
axs[0][0].set_ylabel("VMT of MUDs' Residents",fontdict=font)

for ax in axs[1]:
    ax.yaxis.grid(True)
    ax.set_xticks([y + 1 for y in range(len(all_data_F))])
    
    ax.set_xlabel('Time of Day',fontdict=font)
axs[1][0].set_ylabel("VMT of Others' Residents",fontdict=font)
    
# add x-tick labels


plt.setp(axs, xticks=[y + 1 for y in range(len(all_data))],
         xticklabels=range(1,25))

plt.show()

In [ ]:
tod = []
for i in range(24):
    tod.append(str(i) + ':00 to ' + str(i+1) + ':00')
    
col_1v_list = tod
stat_avg_1v_vmt = pd.DataFrame(columns=col_1v_list,index = ['mean of MUDs', 'std of MUDs', 'mean of Others','std of Others','mean of Alls','std of Alls'])
stat_avg_1v_vmt

In [ ]:
for i in range(24):
    stat_avg_1v_vmt.iloc[0,i] = df_avg_1v_vmt_T[i].mean()
    stat_avg_1v_vmt.iloc[1,i] = df_avg_1v_vmt_T[i].std()
    stat_avg_1v_vmt.iloc[2,i] = df_avg_1v_vmt_F[i].mean()
    stat_avg_1v_vmt.iloc[3,i] = df_avg_1v_vmt_F[i].std()
    stat_avg_1v_vmt.iloc[4,i] = df_avg_1v_vmt[i].mean()
    stat_avg_1v_vmt.iloc[5,i] = df_avg_1v_vmt[i].std()
stat_avg_1v_vmt   

In [ ]:
tod = []
for i in range(24):
    tod.append(str(i) + ':00 to ' + str(i+1) + ':00')

plt.figure(figsize =(20,12))

plt.plot(tod, avg_1v_vmt_T[0], label='MUDs');
plt.plot(tod, avg_1v_vmt_F[0], label='Others');
#plt.plot(tod, avg_vmt[0], label='Alls');

plt.xlabel('Time of day', fontdict=font)
plt.ylabel('Average VMT (Miles)', fontdict=font)
plt.title('Average VMTs of predicted MUDs for one primary vehicle per household', fontsize=28)
plt.legend(loc='upper right')
plt.xticks(rotation=45)

plt.show()

In [ ]:
trip17_MUD_1v_vmt_T = trip17_MUD_1v_vmt[trip17_MUD_1v_vmt['MUDs_Predict_OW'] == 1]
trip17_MUD_1v_vmt_F = trip17_MUD_1v_vmt[trip17_MUD_1v_vmt['MUDs_Predict_OW'] == 0]
trip17_MUD_1v_vmt_T = trip17_MUD_1v_vmt_T[
    np.abs(trip17_MUD_1v_vmt_T.VMT_MILE - trip17_MUD_1v_vmt_T.VMT_MILE.mean()) <= (3*trip17_MUD_1v_vmt_T.VMT_MILE.std())
]

trip17_MUD_1v_vmt_F = trip17_MUD_1v_vmt_F[
    np.abs(trip17_MUD_1v_vmt_F.VMT_MILE - trip17_MUD_1v_vmt_F.VMT_MILE.mean()) <= (3*trip17_MUD_1v_vmt_F.VMT_MILE.std())
]
f, axes = plt.subplots(2, 1, figsize=(20, 16), sharex=True)
#plt.figure(figsize =(20,12))

sns.distplot(trip17_MUD_1v_vmt_T['VMT_MILE'],ax=axes[0]);
axes[0].set(xlabel='VMT (mile)', ylabel="Density", title = 'VMT of MUDs')

sns.distplot(trip17_MUD_1v_vmt_F['VMT_MILE'],ax=axes[1],color="g");
axes[1].set(xlabel='VMT (mile)', ylabel="Density", title = 'VMT of Others')
#axes.legend(loc='upper right');
f.suptitle('Distribution of Trip distance in miles for one primary vehicle per household', fontsize=24);


In [ ]:

trip17_MUD_1v_vmt_T = trip17_MUD_1v_vmt[trip17_MUD_1v_vmt['MUDs_Predict_OW'] == 1]
trip17_MUD_1v_vmt_F = trip17_MUD_1v_vmt[trip17_MUD_1v_vmt['MUDs_Predict_OW'] == 0]


trip17_MUD_1v_vmt_T = trip17_MUD_1v_vmt_T[
    np.abs(trip17_MUD_1v_vmt_T.VMT_MILE - trip17_MUD_1v_vmt_T.VMT_MILE.mean()) <= (3*trip17_MUD_1v_vmt_T.VMT_MILE.std())
]
trip17_MUD_1v_vmt_T = trip17_MUD_1v_vmt_T.groupby('VMT_MILE').WTTRDFIN.sum().reset_index(name ='count')
trip17_MUD_1v_vmt_T = trip17_MUD_1v_vmt_T.groupby(pd.cut(trip17_MUD_1v_vmt_T.VMT_MILE, bins=np.arange(trip17_MUD_1v_vmt_T.VMT_MILE.min(), trip17_MUD_1v_vmt_T.VMT_MILE.max(), step=10),include_lowest=True)).sum()
trip17_MUD_1v_vmt_T = trip17_MUD_1v_vmt_T.append({'VMT_MILE': 0, 'count':0}, ignore_index=True)
trip17_MUD_1v_vmt_T = trip17_MUD_1v_vmt_T.append({'VMT_MILE': 0, 'count':0}, ignore_index=True)


trip17_MUD_1v_vmt_F = trip17_MUD_1v_vmt_F[
    np.abs(trip17_MUD_1v_vmt_F.VMT_MILE - trip17_MUD_1v_vmt_F.VMT_MILE.mean()) <= (3*trip17_MUD_1v_vmt_F.VMT_MILE.std())
]
trip17_MUD_1v_vmt_F = trip17_MUD_1v_vmt_F.groupby('VMT_MILE').WTTRDFIN.sum().reset_index(name ='count')
trip17_MUD_1v_vmt_F = trip17_MUD_1v_vmt_F.groupby(pd.cut(trip17_MUD_1v_vmt_F.VMT_MILE, bins=np.arange(trip17_MUD_1v_vmt_F.VMT_MILE.min(), trip17_MUD_1v_vmt_F.VMT_MILE.max(), step=10),include_lowest=True)).sum()

VMT_MILE = []
for i in range(10):
    VMT_MILE.append(str(10*i) + ' to ' + str(10*(i+1)))

x = np.arange(len(VMT_MILE))  # the label locations
width = 0.35       # the width of the bars: can also be len(x) sequence

fig, ax = plt.subplots(figsize = (16,8))

# rects1 = ax.bar(x - width/2, men_means, width, label='Men')
# rects2 = ax.bar(x - width/2, women_means, width, label='Women')

# plt.bar(VMT_MILE, trip17_MUD_vmt_T['count'],x - width/2, width, label='MUDs');
# plt.bar(VMT_MILE, trip17_MUD_vmt_F['count'],x + width/2, width, label='Others');

ax.bar(x - width/2, trip17_MUD_1v_vmt_T['count'], width, label='MUDs');
ax.bar(x + width/2, trip17_MUD_1v_vmt_F['count'], width, label='Others');

ax.set_xlabel('VMT_MILE(miles)', fontdict=font)
ax.set_ylabel('Counts of trips', fontdict=font)
ax.set_title('Distribution of counts of Trip distance in miles for one primary vehicle per household', fontdict=font)
ax.legend(loc='upper right')
#ax.xticks(rotation=45)
ax.set_xticks(x)
ax.set_xticklabels(VMT_MILE)

plt.show()

% of time 1 primary vehicle staying at home

In [ ]:
trip17_MUD_1v.head()

TRVLCMIN: Trip Duration in Minutes

WHYTRP1S: Trip purpose summary, 01=Home

DWELTIME: Time at destination, -9=Not ascertained

In [ ]:
hsid = trip17_MUD_1v.HOUSEID.unique().tolist()

In [ ]:
trip17_MUD_1v[(trip17_MUD_1v['HOUSEID'] == hsid[0]) &
                          (trip17_MUD_1v['WHYTRP1S'] != '01')]['DWELTIME'].sum()

In [ ]:
trip17_MUD_1v_T = trip17_MUD_1v[trip17_MUD_1v['MUDs_Predict_OW'] == 1]
trip17_MUD_1v_F = trip17_MUD_1v[trip17_MUD_1v['MUDs_Predict_OW'] == 0]

In [ ]:
# MUDs: time not at home

In [ ]:
time_not_home = []
for i in range(len(hsid)):
    temp1 = trip17_MUD_1v_T[trip17_MUD_1v_T['HOUSEID'] == hsid[i]]['TRVLCMIN'].sum()
    temp2 = trip17_MUD_1v_T[(trip17_MUD_1v_T['HOUSEID'] == hsid[i]) &
                          (trip17_MUD_1v_T['WHYTRP1S'] != '01')]['DWELTIME'].sum()
    temp = temp1 + temp2
    time_not_home.append(temp)

In [ ]:
time_not_home = [i for i in time_not_home if i != 0]
time_home_muds = []
for i in range(len(time_not_home)):
    temp = (1440-time_not_home[i])/1440
    time_home_muds.append(temp)


In [ ]:
# Others: time not at home

In [ ]:
time_not_home_others = []
for i in range(len(hsid)):
    temp1 = trip17_MUD_1v_F[trip17_MUD_1v_F['HOUSEID'] == hsid[i]]['TRVLCMIN'].sum()
    temp2 = trip17_MUD_1v_F[(trip17_MUD_1v_F['HOUSEID'] == hsid[i]) &
                          (trip17_MUD_1v_F['WHYTRP1S'] != '01')]['DWELTIME'].sum()
    temp = temp1 + temp2
    time_not_home_others.append(temp)

In [ ]:
time_not_home_others = [i for i in time_not_home_others if i != 0]
time_home_others = []
for i in range(len(time_not_home_others)):
    temp = (1440-time_not_home_others[i])/1440
    time_home_others.append(temp)

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True,figsize = (12,8))
n_bins = 20
# We can set the number of bins with the `bins` kwarg
axs[0].hist(time_home_muds, bins=n_bins)
axs[1].hist(time_home_others, bins=n_bins)
axs[0].set_title('distribution of % time staying at home of MUDs')
axs[1].set_title('distribution of % time staying at home of Others')
axs[0].set_xlim(0, 1);
axs[1].set_xlim(0, 1);
plt.show()

In [ ]:
##### box plot y:0-1;;;

In [ ]:
def dist_share_home(time_home_muds,time_home_others,division):
    fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True,figsize = (12,8))
    n_bins = 20
    # We can set the number of bins with the `bins` kwarg
    axs[0].hist(time_home_muds, bins=n_bins)
    axs[1].hist(time_home_others, bins=n_bins)
    axs[0].set_title('distribution of % time staying at home of MUDs of '+ division)
    axs[1].set_title('distribution of % time staying at home of Others of '+ division)
    axs[0].set_xlim(0, 1);
    axs[1].set_xlim(0, 1);
    plt.show()

In [ ]:
division_list = ['New England','Middle Atlantic','East North Central','West North Central',
                 'South Atlantic','East South Central','West South Central','Mountain','Pacific']

for i in range(9):
    division = division_list[i]
    trip17_MUD_CENSUS = trip17_MUD_1v[trip17_MUD_1v['CENSUS_D'] == i+1]
    trip17_MUD_1v_T = trip17_MUD_CENSUS[trip17_MUD_CENSUS['MUDs_Predict_OW'] == 1]
    trip17_MUD_1v_F = trip17_MUD_CENSUS[trip17_MUD_CENSUS['MUDs_Predict_OW'] == 0]

    time_not_home = []
    for i in range(len(hsid)):
        temp1 = trip17_MUD_1v_T[trip17_MUD_1v_T['HOUSEID'] == hsid[i]]['TRVLCMIN'].sum()
        temp2 = trip17_MUD_1v_T[(trip17_MUD_1v_T['HOUSEID'] == hsid[i]) &
                          (trip17_MUD_1v_T['WHYTRP1S'] != '01')]['DWELTIME'].sum()
        temp = temp1 + temp2
        time_not_home.append(temp)
    time_not_home = [i for i in time_not_home if i != 0]
    time_home_muds = []
    for i in range(len(time_not_home)):
        temp = (1440-time_not_home[i])/1440
        time_home_muds.append(temp)

        
    time_not_home_others = []
    for i in range(len(hsid)):
        temp1 = trip17_MUD_1v_F[trip17_MUD_1v_F['HOUSEID'] == hsid[i]]['TRVLCMIN'].sum()
        temp2 = trip17_MUD_1v_F[(trip17_MUD_1v_F['HOUSEID'] == hsid[i]) &
                              (trip17_MUD_1v_F['WHYTRP1S'] != '01')]['DWELTIME'].sum()
        temp = temp1 + temp2
        time_not_home_others.append(temp)
    time_not_home_others = [i for i in time_not_home_others if i != 0]
    time_home_others = []
    for i in range(len(time_not_home_others)):
        temp = (1440-time_not_home_others[i])/1440
        time_home_others.append(temp)
    
    dist_share_home(time_home_muds,time_home_others,division)

In [ ]:
#### 
trip2 = trip17.groupby('TRPTRANS').WTTRDFIN.sum().reset_index(name ='count')
trip2.head()

In [ ]:
# Trip Mode, deriv
trip2 = trip17.groupby('TRPTRANS').WTTRDFIN.sum().reset_index(name ='count')
trip2['proportion(%)'] = 100*trip2['count']/trip2['count'].sum()
trip2=trip2.sort_values(by='proportion(%)', ascending=False)
trip2.plot(x='TRPTRANS', y='proportion(%)', kind='bar',figsize=(16,10),title='Trip Mode',fontsize=14);
plt.xticks(rotation=45)

In [ ]:
Image(filename="img/trptans.png",width=800, height=800)

In [ ]:
#Only consider trips with personal vehicles
# exclude taxi
vehs = ['3','4','5','6','7','8','17','18']
trip17 = trip17.loc[trip17['TRPTRANS'].isin(vehs)]

In [ ]:
#tripday.loc[tripday.TRAVDAY =='1','TRAVDAY'] = 'Sunday'
tripday = trip17.groupby('TRAVDAY').WTTRDFIN.sum().reset_index(name ='count')
tripday['TRAVDAY'] = tripday['TRAVDAY'].map({1:'Sunday',2:'Monday',3:'Tuesday',4:'Wednesday',5:'Thursday',6:'Friday',7:'Saturday'})
tripday.head(20)
tripday['proportion(%)'] = 100*tripday['count']/tripday['count'].sum()
tripday=tripday.sort_values(by='proportion(%)', ascending=False)
tripday.plot(x='TRAVDAY', y='proportion(%)', kind='bar',figsize=(16,10),title='Trips of Day of week',fontsize=14);
plt.xticks(rotation=45)

In [ ]:
# Travel day - day of week
tripday = trip17.groupby('TRAVDAY').WTTRDFIN.sum().reset_index(name ='count')
# tripday[tripday.TRAVDAY =='1'] = 'Sunday'
# tripday['proportion(%)'] = 100*tripday['count']/tripday['count'].sum()
# tripday=tripday.sort_values(by='proportion(%)', ascending=False)
# tripday.plot(x='TRAVDAY', y='proportion(%)', kind='bar',figsize=(16,10),title='Trips of Day of week',fontsize=14);

In [ ]:
Image(filename="img/travday.png",width=100, height=100)

In [ ]:
#Trip distance in miles, derived from route geometry returned
# Remove outliers which are not within 3 standard deviations from mean of trip dataset
trip170 = trip17[trip17['TRPMILES']>0]

trip18 = trip170[
    np.abs(trip170.TRPMILES - trip170.TRPMILES.mean()) <= (3*trip170.TRPMILES.std())
]
tripvmt = trip18.groupby('TRPMILES').WTTRDFIN.sum().reset_index(name ='count')
tripvmt = tripvmt.groupby(pd.cut(tripvmt.TRPMILES, bins=np.arange(tripvmt.TRPMILES.min(), tripvmt.TRPMILES.max(), step=10),include_lowest=True)).sum()
# tripvmt.head()

# ax = tripvmt.plot.hist(bins=[0,10,30,60,120,200],figsize=(16,10),title='Trip distance in miles',fontsize=16)
# x = np.linspace(0, 250, num=26)
# plt.xticks(x)
tripvmt['proportion(%)'] = 100*tripvmt['count']/tripvmt['count'].sum()
tripvmt=tripvmt.sort_values(by='proportion(%)', ascending=False)

#tripvmt.TRPMILES = pd.cut(tripvmt.TRPMILES, bins=np.arange(tripvmt.TRPMILES.min(), tripvmt.TRPMILES.max(), step=10), include_lowest=True)
#tripvmt = tripvmt.groupby(pd.cut(tripvmt.TRPMILES, bins=np.arange(tripvmt.TRPMILES.min(), tripvmt.TRPMILES.max(), step=10),include_lowest=True)).count.sum()
#
tripvmt = tripvmt[['proportion(%)']]
tripvmt.reset_index('TRPMILES')
tripvmt['proportion(%)'].plot(kind='bar',figsize=(16,10),title='Trip distance in miles',fontsize=14);
plt.xticks(rotation=45)

In [ ]:
plt.style.use("fivethirtyeight")

trip17_MUD_True_1 = trip17_MUD_True[trip17_MUD_True['TRPMILES']>0]

trip17_MUD_True_1 = trip17_MUD_True_1[
    np.abs(trip17_MUD_True_1.TRPMILES - trip17_MUD_True_1.TRPMILES.mean()) <= (3*trip17_MUD_True_1.TRPMILES.std())
]
trip17_MUD_True_1 = trip17_MUD_True_1.groupby('TRPMILES').WTTRDFIN.sum().reset_index(name ='count')
trip17_MUD_True_1 = trip17_MUD_True_1.groupby(pd.cut(trip17_MUD_True_1.TRPMILES, bins=np.arange(trip17_MUD_True_1.TRPMILES.min(), trip17_MUD_True_1.TRPMILES.max(), step=10),include_lowest=True)).sum()
trip17_MUD_True_1 = trip17_MUD_True_1.head(22)

width = 0.35       # the width of the bars: can also be len(x) sequence
plt.figure(figsize =(20,12))

plt.bar(TRPMILES, trip17_MUD_True_1['count'], width, label='MUDs')
plt.bar(TRPMILES, trip17_MUD_False_1['count'], width, label='Others',bottom=np.array(trip17_MUD_True_1['count']) )

plt.xlabel('TRPMILES(miles)')
plt.ylabel('Counts')
plt.title('Trip distance in miles')
plt.legend(loc='upper right')
plt.xticks(rotation=45)

plt.show()

In [ ]:
print(len(TRPMILES))
print(len(trip17_MUD_True_1['count']))
len(trip17_MUD_False_1['count'])

In [ ]:
trip17_MUD_True_1.head()

In [ ]:
plt.style.use("seaborn")

In [ ]:
trip17_MUD_False_1 = trip17_MUD_False[trip17_MUD_False['TRPMILES']>0]

trip17_MUD_False_1 = trip17_MUD_False_1[
    np.abs(trip17_MUD_False_1.TRPMILES - trip17_MUD_False_1.TRPMILES.mean()) <= (3*trip17_MUD_False_1.TRPMILES.std())
]
trip17_MUD_False_1 = trip17_MUD_False_1.groupby('TRPMILES').WTTRDFIN.sum().reset_index(name ='count')
trip17_MUD_False_1 = trip17_MUD_False_1.groupby(pd.cut(trip17_MUD_False_1.TRPMILES, bins=np.arange(trip17_MUD_False_1.TRPMILES.min(), trip17_MUD_False_1.TRPMILES.max(), step=10),include_lowest=True)).sum()


In [ ]:
trip17_MUD_False_1.head()

In [ ]:
# Trip Origin Purpose
tripfrom = trip17.groupby('WHYFROM').WTTRDFIN.sum().reset_index(name ='count')
tripfrom['proportion(%)'] = 100*tripfrom['count']/tripfrom['count'].sum()
tripfrom=tripfrom.sort_values(by='proportion(%)', ascending=False)
tripfrom.plot(x='WHYFROM', y='proportion(%)',kind='bar',figsize=(16,10),title='Trip Origin Purpose',fontsize=14);
plt.xticks(rotation=45)

In [ ]:
Image(filename="img/whyfrom.png",width=800, height=800)

In [ ]:
# Trip Destination Purpose
trip1 = trip17.groupby('WHYTO').WTTRDFIN.sum().reset_index(name ='count')
trip1['proportion(%)'] = 100*trip1['count']/trip1['count'].sum()
trip1=trip1.sort_values(by='proportion(%)', ascending=False)
trip1.plot(x='WHYTO', y='proportion(%)',kind='bar',figsize=(16,10),title='Trip Destination Purpose',fontsize=14);
plt.xticks(rotation=45)

In [ ]:
Image(filename="img/whyto.png",width=800, height=800)

In [ ]:
# Trip Purpose Summary
trip0 = trip17.groupby('WHYTRP1S').WTTRDFIN.sum().reset_index(name ='count')
trip0['proportion(%)'] = 100*trip0['count']/trip0['count'].sum()
trip0=trip0.sort_values(by='proportion(%)', ascending=False)
trip0.plot(x='WHYTRP1S', y='proportion(%)',kind='bar',figsize=(16,10),title='Trip Purpose Summary',fontsize=14);
plt.xticks(rotation=45)

In [ ]:
plt.style.use("seaborn")
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 16,
        }
#home & work & public charging
chargers = {'AggWHYTRP1S': ['Home','Work','Public'],
        'prop': [trip0.iloc[0,2],trip0.iloc[2,2],100-trip0.iloc[0,2]- trip0.iloc[2,2]]
        }

df = pd.DataFrame(chargers, columns = ['AggWHYTRP1S', 'prop'])
df
df.plot(x='AggWHYTRP1S', y='prop',kind='bar',figsize=(15,9), legend=None)
plt.xticks(rotation=45)
plt.title('Genral Trip Purpose Shares', fontdict=font)
plt.xlabel('Genral trip origins', fontdict=font)
plt.ylabel('Share (%)', fontdict=font)

In [ ]:
trip17['WHYTRP1S'].dtype

In [ ]:
## calculate VMTs for three trips locations: home, work, public
VMT_home = trip17[trip17['WHYTRP1S'] == 1]['VMT_MILE'].sum()
VMT_work = trip17[trip17['WHYTRP1S'] == 10]['VMT_MILE'].sum()
VMT_public = trip17['VMT_MILE'].sum() - VMT_home - VMT_work


In [ ]:
from matplotlib import pyplot as plt

plt.style.use("seaborn")

slices = [VMT_home, VMT_work, VMT_public]
labels = ['Home(MUDs & others)', 'Work', 'Public']
explode = [0.1, 0, 0]

plt.pie(slices, labels=labels, explode=explode, shadow=True,
        startangle=90, autopct='%1.1f%%',
        wedgeprops={'edgecolor': 'black'})

plt.title("VMTs")
plt.tight_layout()
plt.show()

In [ ]:
Image(filename="img/WHYTRP1S.png",width=1000, height=1000)

### Shall we consider Taxi, rental car... into personal vehicles?

In [ ]:
plt.style.use("seaborn")
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 16,
        }
#trip time of each trip with vehicles
vehs = ['3','4','5','6','7','8','17','18']
trip3 = trip17.loc[trip17['TRPTRANS'].isin(vehs)]
trip3['triptime'] = trip3.ENDTIME - trip3.STRTTIME

trip3 = trip3[trip3['triptime']>0]

trip3 = trip3[
    np.abs(trip3.triptime - trip3.triptime.mean()) <= (3*trip3.triptime.std())
]

trip4 = trip3.groupby('triptime').WTTRDFIN.sum().reset_index(name ='count')
trip4 = trip4.groupby(pd.cut(trip4.triptime, bins=np.arange(trip4.triptime.min(), trip4.triptime.max(), step=20),include_lowest=True)).sum()
# tripvmt.head()
trip4['proportion(%)'] = 100*trip4['count']/trip4['count'].sum()

trip4 = trip4[['proportion(%)']]
trip4.reset_index('triptime')
trip4['proportion(%)'].plot(kind='bar',figsize=(15,9))

plt.xticks(rotation=45)
plt.title('Trip time', fontdict=font)
plt.xlabel('Trip time (min)', fontdict=font)
plt.ylabel('Share (%)', fontdict=font)
# trip4 = trip3.groupby('triptime').WTTRDFIN.sum().reset_index(name ='count')
# # trip4['proportion(%)'] = 100*trip4['count']/trip4['count'].sum()
# # trip4=trip4.sort_values(by='proportion(%)', ascending=False)

# # trip4.plot(x='triptime', y='proportion(%)',kind='bar',figsize=(16,10),title='trip time of each trip',fontsize=14);
# ax = trip4.plot.hist(bins=np.arange(0, 2880, 50),figsize=(16,10),title='trip time of each trip',fontsize=16)
# #trip4.plot(x='triptime', y='count',kind='bar',figsize=(16,10),title='trip time of each trip');

In [ ]:
np.arange(0, 2880, 60)

In [ ]:
#Dwelling time between trips
trip5 = trip17.loc[trip17.TRPTRANS.isin(vehs)]

trip5 = trip5[trip5['DWELTIME']>0]

trip5 = trip5[
    np.abs(trip5.DWELTIME - trip5.DWELTIME.mean()) <= (3*trip5.DWELTIME.std())
]


trip5 = trip5.groupby('DWELTIME').WTTRDFIN.sum().reset_index(name ='count')
trip5 = trip5.groupby(pd.cut(trip5.DWELTIME, bins=np.arange(trip5.DWELTIME.min(), trip5.DWELTIME.max(), step=20),include_lowest=True)).sum()
# tripvmt.head()
trip5['proportion(%)'] = 100*trip5['count']/trip5['count'].sum()
trip5=trip5.sort_values(by='proportion(%)', ascending=False)

trip5 = trip5[['proportion(%)']]
trip5.reset_index('DWELTIME')
trip5 = trip5.iloc[::-1]
trip5['proportion(%)'].plot(kind='barh',figsize=(16,10),)
plt.title('Dwelling time', fontdict=font)
plt.ylabel('DWELTIME(min)', fontdict=font)
plt.xlabel('Share (%)', fontdict=font)


# trip5 = trip5.groupby('DWELTIME').WTTRDFIN.sum().reset_index(name ='count')
# # trip5['proportion(%)'] = 100*trip5['count']/trip5['count'].sum()
# # trip5=trip5.sort_values(by='proportion(%)', ascending=False)

# #trip5.plot(x=np.arange(0, 2880, 100), y='proportion(%)',kind='bar',figsize=(16,10),title='Dwelling time between trips',fontsize=14);
# ax = trip5.plot.hist(bins=np.arange(0, 2880, 60),figsize=(16,10),title='Dwelling time between trips',fontsize=16)

In [ ]:
trip5.DWELTIME

In [ ]:
dictionary17 = pd.read_excel('NHTS-daily travel patterns/dictionary_v1_2.xlsx')
dictionary17

In [ ]:
codebook17 = pd.read_excel('NHTS-daily travel patterns/codebook_v1.2.xlsx')
codebook17.head()

In [ ]:
trip17.describe()

In [ ]:
# Use "UnitInStructe" dataset to make assumption about household type of MUD or single family.
pd.set_option('display.max_columns', None)
single_MUD = pd.read_excel('UnitInStructure.xlsx', sheet_name = 'MUD', index_col=0)
single_MUD['Total'] = single_MUD.sum(axis=1) 
single_MUD.info()

In [ ]:
# Assume "1, detached" and "1, attached" are single family house.
# So, column 2 to 6 can be Multi-Unit Dwelling (i.e. Apartment)
single_MUD['MUD #'] = single_MUD.iloc[:, 2:6].sum(axis=1) 
single_MUD['MUD percentage'] = single_MUD['MUD #']/single_MUD['Total']
single_MUD

In [ ]:
single_MUD['MUD percentage'].plot(kind='bar',figsize=(16,10),title='MUD percentage');
plt.xticks(rotation=45)

In [ ]:
# research garage among household
# Year 2015 - 2019
pd.set_option('display.max_columns', None)
garage1519 = pd.read_excel('UnitInStructure.xlsx', sheet_name = 'Garage1519', index_col=0)
garage1519

In [ ]:
garage1519.plot(kind='bar',figsize=(16,10),title='Garage info 2015 - 2019');
plt.xticks(rotation=45)

In [ ]:
# research garage among household
# Year 2013
pd.set_option('display.max_columns', None)
garage13 = pd.read_excel('UnitInStructure.xlsx', sheet_name = 'Garage13', index_col=0)
garage13

In [ ]:
garage13.plot(kind='bar',figsize=(16,10),title='Garage info 2013');
plt.xticks(rotation=45)

In [ ]:
# research garage among household
# Year 2011
pd.set_option('display.max_columns', None)
garage11 = pd.read_excel('UnitInStructure.xlsx', sheet_name = 'Garage11', index_col=0)
garage11

In [ ]:
#Trip Distance sample size and weighted.
tripDist = pd.read_excel('UnitInStructure.xlsx', sheet_name = 'TripDistance', index_col=0)
tripDist['Percent WT'] = 100*tripDist['Sum (Millions)']/tripDist.loc['All','Sum (Millions)']
tripDist['Percent Sample'] = 100*tripDist['Sample Size']/tripDist.loc['All','Sample Size']
tripDist

In [ ]:
tripDistPlt = tripDist.loc[:,['Percent Sample','Percent WT']]
tripDistPlt = tripDistPlt[:-1]
tripDistPlt.plot(kind='bar',figsize=(16,10),title='Trip Distance with Sample and Weighted Sum');
plt.xticks(rotation=45)

In [ ]:
#VMT corresponding to ages sample size and weighted.
vmtAge = pd.read_excel('UnitInStructure.xlsx', sheet_name = 'VMT_AGE', index_col=0)
vmtAge['Percent WT'] = 100*vmtAge['Sum (Millions)']/vmtAge.loc['All','Sum (Millions)']
vmtAge['Percent Sample'] = 100*vmtAge['Sample Size']/vmtAge.loc['All','Sample Size']
vmtAge

In [ ]:
vmtAgePlt = vmtAge[:-1]
vmtAgePlt.loc[:,['Percent Sample','Percent WT']].plot(kind='bar',figsize=(16,10),title='VMT ~ Age with Sample and Weighted Sum');
plt.xticks(rotation=45)

In [ ]:
#VMT corresponding to Destinations sample size and weighted.
vmtDes = pd.read_excel('UnitInStructure.xlsx', sheet_name = 'VMT_Destination', index_col=0)
vmtDes['Percent WT'] = 100*vmtDes['Sum (Millions)']/vmtDes.loc['All','Sum (Millions)']
vmtDes['Percent Sample'] = 100*vmtDes['Sample Size']/vmtDes.loc['All','Sample Size']
vmtDes

In [ ]:
vmtDesPlt = vmtDes[:-1]
vmtDesPlt.loc[:,['Percent Sample','Percent WT']].plot(kind='bar',figsize=(16,10),title='Time at Destination Sample and Weighted Sum');
plt.xticks(rotation=45)

In [ ]:
#Vehicle Trips corresponding to Dwelling time sample size and weighted.
VtripDwell = pd.read_excel('UnitInStructure.xlsx', sheet_name = 'VTrip_Dwell', index_col=0)
VtripDwell['Percent WT'] = 100*VtripDwell['Sum (Millions)']/VtripDwell.loc['All','Sum (Millions)']
VtripDwell['Percent Sample'] = 100*VtripDwell['Sample Size']/VtripDwell.loc['All','Sample Size']
VtripDwell

In [ ]:
VtripDwellPlt = VtripDwell[:-1]
VtripDwellPlt.loc[:,['Percent Sample','Percent WT']].plot(kind='bar',figsize=(16,10),title='Time at Destination with Sample and Weighted Sum');
plt.xticks(rotation=45)

#### half is over 50+ min, which shows the potential to provide charger station at the destination during the parking.